In [1]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import cv2
from keras.models import load_model


Using TensorFlow backend.


In [2]:

batch_size = 512
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()


In [ ]:
t = 57
cv2.imshow("train sample", x_train[t])
print(y_train[t])
cv2.waitKey(0)
cv2.destroyAllWindows()

9


In [11]:
img = cv2.imread('5.png')
img.shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [3]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

print(y_train.shape)
print(y_test.shape)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
(60000, 10)
(10000, 10)


In [4]:
x_train[0].shape

(28, 28, 1)

In [5]:

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])



In [11]:
model.fit(x_train, y_train,
          batch_size=1024,
          epochs=epochs,
          5
          verbose=1,
          validation_data=(x_test, y_test))


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
model.save("model_v3.h5")

SyntaxError: invalid syntax (<ipython-input-11-b580301796d6>, line 5)

In [6]:
model = load_model('model_v3.h5')

In [7]:
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
__________

In [8]:
import cv2

In [ ]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    rrec = cv2.rectangle(frame,(300,200),(400,300),(255,255,255,5))
    
    # Display the resulting frame
   # cv2.imshow('frame',gray)

    #rrec_frame = ('rrec',gray_rrec)
    
    
    crop_cap = gray[200:300,300:400]
    text = recognition(crop_cap)
    cv2.putText(rrec,text,(100,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(0,0,255),1,cv2.LINE_AA)
    cv2.imshow('test_image',rrec)
    cv2.imshow('crop_image',crop_cap)
    
    #print(resized_img.shape)
   
    #img_path = cap

    #img = 
    #img = img.reshape(1,28,28,1)
    #img = img.astype('float32')
    #img /= 255
    
    #preds = model.predict(img)

    #labels = [0,1,2,3,4,5,6,7,8,9]
    #ind = preds[0].argmax()
    #print("Ответ : ",labels[ind], ", точность :", round(preds[0][ind]*100,2),"%" )
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
    

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

In [9]:
def recognition(img):
    resized_img = cv2.resize(img, (28, 28)) 
    img = resized_img.reshape(1,28,28,1)
    img = img.astype('float32')
    img /= 255
    preds = model.predict(img)
    labels = [0,1,2,3,4,5,6,7,8,9]
    ind = preds[0].argmax()
    #print("Ответ : ",labels[ind], ", точность :", round(preds[0][ind]*100,2),"%" )
    acc = round(preds[0][ind]*100,2)
    if acc > 75:
        res = ""+str(labels[ind])+", "+str(acc) +"%"
    else:
        res= "0, 0%"
    return res